# Project UTS - Pengolahan Bahasa Alami
##  KLASIFIKASI TEKS BAHASA INDONESIA UNTUK ...

**Anggota Kelompok:**
1. Stefanus Vemas Aditya Mahardika - 210711398
2. Alexis Divasonda Sigat Ngaing - 210711407
3. Dave Sebastian Petrus - 210711487
4. Tio Pramudya - 210711411

## Objective
Silahkan isi bagian ini dengan dengan objektif/tujuan dari ide projek kelompok Anda.

## Latar Belakang
Silahkan isi bagian ini dengan dengan latar belakang dari ide projek kelompok Anda.

## Deskripsi Dataset
Silahkan isi bagian ini dengan sumber dataset untuk projek kelompok Anda, bagaimana kelompok Anda mengumpulkan data, dan karakteristik dari dataset kelompok Anda, misal jumlah untuk masing-masing label.

## Pengembangan Model Klasifikasi Teks


### Inisialisasi
Silakan isi bagian ini dengan mengimport library-library yang akan digunakan. 

In [46]:
import pandas as pd
import numpy as np
import nltk
from contractions_id import CONTRACTION_MAP
import modSpellChecker as sc
# from contractions_id import CONTRACTION_MAP_ID
import re
import string
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import gensim
from gensim.models import Word2Vec
from gensim import models
#from pattern.en import tag
from nltk.corpus import wordnet as wn
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from gensim import corpora, models
import sklearn
#from normalization import normalize_corpus
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

### Data Preparation
Silakan isi bagian ini dengan script untuk memanggil data yang akan digunakan untuk keperluan pengembangan model klasifikasi teks dan menampilkan contoh data yang telah dipanggil baik yang akan dijadikan sebagai input maupun target-nya (label).

In [47]:
dataset = pd.read_csv('merapi.csv')
dataset

,name,text
0,Melisa Aritonang,Masih berada di kaki gunung\nSeru sekali bisa ...
1,Irsya Kumala,"Amazing ..\nSangat Suka,,. pertama kali kesini..."
2,Dewi Natalia,Ke merapi naik jeep tahun 2016 seru banget ka...
3,Fajar Aqil,Banyak pemandangan sangat indah yang bisa dile...
4,wawan budiana wikanda,Sebuah pengalaman yg tak akan terulang 2x\nPen...
...,...,...
995,Hiren Patel,NaN
996,Reynaldi pramudya,NaN
997,Douglas Lee-OLoughlin,NaN
998,Retno Tri Utami,NaN


In [48]:
dataset.shape

(1000, 2)

In [49]:
dataset = dataset.dropna()
dataset

,name,text
0,Melisa Aritonang,Masih berada di kaki gunung\nSeru sekali bisa ...
1,Irsya Kumala,"Amazing ..\nSangat Suka,,. pertama kali kesini..."
2,Dewi Natalia,Ke merapi naik jeep tahun 2016 seru banget ka...
3,Fajar Aqil,Banyak pemandangan sangat indah yang bisa dile...
4,wawan budiana wikanda,Sebuah pengalaman yg tak akan terulang 2x\nPen...
...,...,...
901,Retno Aja,WAAAAWW ISMEZIK
902,Bijoy Sorkar,Piculiar...
903,葉小良,Sering melutus
904,Robbydwi Riski,Yuhu


In [50]:
label = dataset.iloc[:,0]
feature = dataset.iloc[:,1]

In [51]:
label[0:10]

0         Melisa Aritonang
1             Irsya Kumala
2             Dewi Natalia
3               Fajar Aqil
4    wawan budiana wikanda
5                mus takim
6           joseph raharjo
7             Hana Uswatun
8       Yuliahan Nariswari
9           Adinda Maulina
Name: name, dtype: object

In [52]:
feature[0:10]

0    Masih berada di kaki gunung\nSeru sekali bisa ...
1    Amazing ..\nSangat Suka,,. pertama kali kesini...
2    Ke merapi naik jeep  tahun 2016 seru banget ka...
3    Banyak pemandangan sangat indah yang bisa dile...
4    Sebuah pengalaman yg tak akan terulang 2x\nPen...
5    Gunung Merapi..\nMerupakan Gunung teraktif did...
6    Gunung merapi mencerminkan kebesaran Tuhan yan...
7    Amazing banget, waktu pertama kali kesana udah...
8    View pagi nya wlwpun berkabut bener bener bagu...
9    My all time favorite volcano. A 5 star rating ...
Name: text, dtype: object

### Normalisasi Data
Silakan isi bagian ini dengan script untuk keperluan normalisasi data teks yang akan digunakan untuk pengembangan model klasifikasi teks. 
Untuk setiap strategi normalisasi yang akan digunakan sebaiknya dibuat fungsi tersendiri dan dipanggil dalam satu fungsi normalisasi. Jangan lupa pada bagian ini ditampilkan contoh hasil normalisasi datanya.

In [53]:
character = ['z','y','x','w','v','u','t','s','r','q','p','o','n','m','l','k','j','i','h','g','f','e','d',
',','.',';',':','-','...','?','!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@']

def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long>=2:
            char=character[i]*charac_long
            text=text.replace(char,character[i])
            charac_long-=1
        return text

def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j=sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
        return spellCheck
    
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def stemmer_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)
    return text

In [54]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = stemmer_text(text)
        text = remove_special_characters(text)
        text = repeatcharNormalize(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            text = spellNormalize(text)
            normalized_corpus.append(text)
    return normalized_corpus

In [55]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels, test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(feature,label,test_data_proportion=0.3)

In [56]:
norm_train_corpus = normalize_corpus(train_corpus)

In [57]:
norm_train_corpus[0:10]

[['sekali',
  'ronde',
  'kesini',
  'yg',
  'jogja',
  'pandemi',
  'nikmat',
  'sudah',
  'sapi'],
 ['gunung', 'at', 'aktif'],
 ['saya', 'cuma', 'sapi', 'cape', 'sinyal', 'yg', 'd', 'lunak'],
 ['marapi', 'at', 'mie', 'zapomnia', 'e', 'widoki'],
 ['mas',
  'di',
  'st',
  'pic',
  'ofrande',
  'fa',
  'tempat',
  'st',
  'cunosc',
  'tehnicile',
  'terapeutice',
  'di',
  'obiectele',
  'cape',
  'ke',
  'beda',
  'atinge',
  'pentru',
  'at',
  'recupera',
  'sate'],
 ['saya', 'sedikit'],
 ['yg',
  'j',
  'mau',
  'i',
  'i',
  'gvjvuvunimininycivu',
  'ynononyv',
  'y',
  'tvrct',
  'yuk',
  'uvtbu',
  'by',
  'h',
  'y',
  'by',
  'ubyhhuh',
  'hhjj',
  'fa',
  'h',
  'hvuvg',
  'yvuvggy',
  'ubinibhnjbjbjbibukanij',
  'di'],
 ['growing',
  'at',
  'watching',
  'sapi',
  'volcanic',
  'mountain',
  'spewing',
  'ke',
  'and',
  'java',
  'from',
  'by',
  'at',
  'second',
  'floor',
  'balcony'],
 ['itu',
  'aktif',
  'suasana',
  'candid',
  'dari',
  'at',
  'tinggal',
  'by',


In [58]:
norm_test_corpus = normalize_corpus(test_corpus)

In [59]:
norm_test_corpus[0:10]

[['gunung',
  'sapi',
  'yg',
  'gaya',
  'pesona',
  'yg',
  'baru',
  'naik',
  'statesnya',
  'siapa',
  'mal',
  'temen',
  'baru',
  'tantang',
  'sendiri',
  'wisatawan',
  'yg',
  'unjung'],
 ['tempat', 'saat', 'sudah', 'kesini'],
 ['apik', 'and'],
 ['gunung',
  'rena',
  'tempat',
  'yg',
  'sudah',
  'sek',
  'senang',
  'daerah',
  'istimewa',
  'jogjakarta',
  'putar',
  'masyarakat',
  'yg',
  'adem',
  'ayam',
  'rame',
  'pikuk',
  'senang',
  'wisata',
  'bareng',
  'gunung',
  'sapi'],
 ['tempat',
  'naik',
  'lagi',
  'dari',
  'rutinitas',
  'kerja',
  'recharge',
  'yuk',
  'and',
  'sore'],
 ['pikuk', 'e', 'tante', 'dan'],
 ['perlu', 'tanya', 'ayam', 'dari', 'sapi'],
 ['lagi',
  'pasar',
  'bareng',
  'sapi',
  'di',
  'sekali',
  'jangan',
  'gunung',
  'bisa',
  'energi',
  'rantau'],
 ['jeanpierre',
  'baru',
  'ke',
  'promener',
  'at',
  'volcan',
  'sapi',
  'jogyakarta',
  'jawa',
  'indonesia',
  'super'],
 ['jalan',
  'sate',
  'gunung',
  'at',
  'malang'

### Ekstraksi Fitur
Silakan isi bagian ini dengan script untuk keperluan ekstraksi fitur data teks yang akan digunakan untuk pengembangan model klasifikasi teks. 
Untuk setiap strategi ekstraksi fitur yang akan digunakan sebaiknya dibuat fungsi tersendiri dan dipanggil dalam satu fungsi ekstraksi fitur. Jangan lupa pada bagian ini ditampilkan contoh hasil ekstraksi fitur terhadap datanya.

### Model Training
Silakan isi bagian ini dengan script untuk keperluan training model klasifikasi teks. Jangan lupa untuk menjabarkan rancangan pemodelan algoritme yang akan diterapkan dan kombinasi parameter yang akan digunakan dengan benar dan lengkap.

### Model Evaluation
Silakan isi bagian ini dengan script untuk keperluan evaluasi model klasifikasi teks. Jangan lupa untuk menjabarkan rancangan proses evaluasi model, metrik, dan visualisasi confusion matrix yang akan digunakan dengan benar dan lengkap

### Model Testing
Silakan isi bagian ini dengan script untuk keperluan testing model terbaik yang telah dihasilkan dengan memberikan input berupa data teks yang sama sekali baru dan belum ada di dataset.